# Feature engineering with RAPIDS on SageMaker Processing

### このノートブックのインスタンスタイプの確認
- このノートブックを実行するインスタンスにはml.t2.large（vCPU2, メモリ8GB）以上のインスタンスタイプを使用してください。ml.t2.medium（vCPU2, メモリ4GB）の場合、コンテナのビルドでメモリエラーが発生します。

### pipとSageMakerの更新
- このnotebookはSageMakerのVersion 2.X 以降で動作するため、SageMakerのVersionが2.0以降でない場合はUpgradeを行う必要があります。
- 下のセルでは2020年11月時点での最新バージョンへ更新を行っています。

In [ ]:
!pip install --upgrade pip
!pip install  sagemaker==2.16.1

### 使用するライブラリのImportとSageMaker, S3の変数設定

In [ ]:
import boto3
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import sagemaker

# SageMakerのVersionが2.0以降であることを確認してください
# もし、upgrade後もVersionが2.0以降に変わらない場合は、カーネルを再起動してください
print('Current SageMaker Python sdk Version ={0}'.format(sagemaker.__version__))

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

prefix = "sagemaker/rapids-preprocess-demo"
input_prefix = prefix + "/input/raw"

### データセットのダウンロードとAmazon Simple Storage Service（Amazon S3）へのアップロード
ここで使用するデータセットは、Census-Income KDDデータセットです。これを使用して、国勢調査の回答者を表す行の収入が50,000ドルより大きいか小さいかを予測するための前処理や特徴量の作成を行います。

In [ ]:
# データのダウンロード
s3 = boto3.client('s3')
region = sagemaker_session.boto_region_name
input_data = 's3://sagemaker-sample-data-{}/processing/census/census-income.csv'.format(region)
!aws s3 cp $input_data .

In [ ]:
# データの読み込みとターゲット変数の変換

df = pd.read_csv('census-income.csv')
print(df.income.value_counts())
df['income'] = np.where(df['income'] == ' 50000+.', 1, 0) #ターゲット変数の(0, 1)変換
print(df.income.value_counts())

df.head()

In [ ]:
# 学習用データと検証用データへの分割

train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['income'])

print(train_df.shape)
print(valid_df.shape)

In [ ]:
# CSVファイルの書き出し
train_df.to_csv('train.csv', index=False)
valid_df.to_csv('validation.csv', index=False)

In [ ]:
# S3へのアップロード
input_train = sagemaker_session.upload_data(path='train.csv', bucket=bucket, key_prefix=input_prefix)
input_validation = sagemaker_session.upload_data(path='validation.csv', bucket=bucket, key_prefix=input_prefix)

print('TrainData is here: ', input_train)
print('validationData is here: ', input_validation)

### dockerコンテナの作成
containerディレクトリの下にあるDockerfileをビルドします。（実行には15分ほどかかります）

さらに必要なライブラリがあれば、Dockerfile内に記述してください。

In [ ]:
%%time

%cd container
!docker build -t sagemaker-rapids-example .
%cd ../

In [ ]:
# Repository URIの設定

account_id = boto3.client('sts').get_caller_identity().get('Account')

ecr_repository = 'sagemaker-rapids-example'
tag = ':latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
rapids_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

print('Your repository URI is: ', rapids_repository_uri)

### Amazon ECR repositoryを作成し、dockerコンテナをPUSHする
上のセルのrepository URIの名前でECRにrepositoryを作成し、sagemaker-rapids-exampleコンテナをPUSHします
（実行には10分前後かかります）

一度コンテナイメージの作成からECRへのPUSHまでを行えば、以降同じコンテナイメージは上のセルで出力されるURIで使用できます。

In [ ]:
%%time

# Create ECR repository and push docker image
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $rapids_repository_uri
!docker push $rapids_repository_uri

### SageMaker Processing内で実行したいコードを記述します。

コンテナの作成が完了したら、コンテナ上で実行したいスクリプトを作成します。今回は、RAPIDSのcuDFとcuMLを使用して、データの読み込みとカテゴリ変数に対するLabelEncoding, TargetEncodingを行い、処理後のファイルを出力した後、XGBoostでモデルの学習を行いたいと思います。

XGBoostのモデルについても、例えば'/opt/ml/processing/output/model'といった名前のディレクトリを作成し、そこへモデルを出力することでS3への保存が可能です。

下のセルを実行するとこのノートブックインスタンスと同じ階層にpreprocess.pyが生成されます。

In [ ]:
%%writefile preprocess.py
from __future__ import print_function, unicode_literals

import boto3
import os
import sys
import time

import cudf
from cuml.preprocessing.LabelEncoder import LabelEncoder
from cuml.preprocessing.TargetEncoder import TargetEncoder

import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")


if __name__ == "__main__":
    
    # Get processor scrip arguments
    args_iter = iter(sys.argv[1:])
    script_args = dict(zip(args_iter, args_iter))
    
    TARGET_COL = script_args['TARGET_COL']
    TE_COLS = [x.strip() for x in script_args['TE_COLS'].split(',')]
    SMOOTH = float(script_args['SMOOTH'])
    SPLIT = script_args['SPLIT']
    FOLDS = int(script_args['FOLDS'])
    
    # Read train, validation data
    train = cudf.read_csv('/opt/ml/processing/input/train/train.csv')
    valid = cudf.read_csv('/opt/ml/processing/input/valid/validation.csv')
    
    start = time.time(); print('Creating Feature...')
    
    # Define categorical columns
    catcols = [x for x in train.columns if x not in [TARGET_COL] and train[x].dtype == 'object']
    
    # Label encoding
    for col in catcols:
        train[col] = train[col].fillna('None')
        valid[col] = valid[col].fillna('None')
        lbl = LabelEncoder()
        lbl.fit(cudf.concat([train[col], valid[col]]))
        train[col] = lbl.transform(train[col])
        valid[col] = lbl.transform(valid[col])
    
    # Target encoding
    for col in TE_COLS:
        out_col = f'{col}_TE'
        encoder = TargetEncoder(n_folds=FOLDS, smooth=SMOOTH, split_method=SPLIT)
        encoder.fit(train[col], train[TARGET_COL])
        train[out_col] = encoder.transform(train[col])
        valid[out_col] = encoder.transform(valid[col])
        
    print('Took %.1f seconds'%(time.time()-start))
        
    print(train.shape)
    print(valid.shape)
    print(train.head())
    print(valid.head())
        
    # Create local output directories
    try:
        os.makedirs('/opt/ml/processing/output/train')
        os.makedirs('/opt/ml/processing/output/valid')
    except:
        pass
    
    # Save data locally
    train.to_csv('/opt/ml/processing/output/train/train.csv', index=False)
    valid.to_csv('/opt/ml/processing/output/valid/validation.csv', index=False)
        
    # Train XGBoost model
    print('XGBoost Version: ',xgb.__version__)
    
    xgb_parms = { 
    'max_depth':6, 
    'learning_rate':0.1, 
    'subsample':0.8,
    'colsample_bytree':1.0, 
    'eval_metric':'auc',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist'
    }
    
    NROUND = 100
    VERBOSE_EVAL = 10
    ESR = 10

    start = time.time(); print('Creating DMatrix...')
    dtrain = xgb.DMatrix(data=train.drop(TARGET_COL,axis=1),label=train[TARGET_COL])
    dvalid = xgb.DMatrix(data=valid.drop(TARGET_COL,axis=1),label=valid[TARGET_COL])
    print('Took %.1f seconds'%(time.time()-start))

    start = time.time(); print('Training...')
    model = xgb.train(xgb_parms, 
                      dtrain=dtrain,
                      evals=[(dtrain,'train'),(dvalid,'valid')],
                      num_boost_round=NROUND,
                      early_stopping_rounds=ESR,
                      verbose_eval=VERBOSE_EVAL)
    
    print('Took %.1f seconds'%(time.time()-start))
    
    print('Finished running processing job')
    

### Processorの定義
ScriptProcessorクラスのimage_uriの引数に、先ほど作成したRAPIDSが実行できるコンテナリポジトリのURIを渡し、インスタンスタイプにGPU（今回はml.p3.2xlarge）を指定し、任意の名前のオブジェクトを作成します。

- ml.p3.2xlargeはNVIDIA Tesla V100が1枚搭載されているインスタンスですが、RAPIDSはマルチGPUにも対応しているため、例えば、NVIDIA Tesla V100が4枚搭載されたp3.8xlargeを使用してpreprocess.pyの中の処理をマルチGPUが実行できるように書き換えることでさらに大規模データへの対応や高速化が実現できます。

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor

rapids_processor = ScriptProcessor(
    role=role, 
    image_uri=rapids_repository_uri,
    command=["python3"],
    instance_count=1, 
    instance_type="ml.p3.2xlarge", # use GPU Instance
    volume_size_in_gb=30, 
    volume_kms_key=None, 
    output_kms_key=None, 
    max_runtime_in_seconds=86400, # the default value is 24 hours(60*60*24)
    base_job_name="rapids-preprocessor",
    sagemaker_session=None, 
    env=None, 
    tags=None, 
    network_config=None)


In [ ]:
# 前処理・特徴量作成したファイルの出力先の定義
output_s3_path = 's3://' + bucket + '/' + prefix + '/input/dataset/'
print('File Output path: ', output_s3_path)

### Processorの実行

下のセルを実行するとpreprocess.pyはS3のsagemaker-{region}-{accountID}の直下に保存されます。

- codeには先ほど作成したpreprocess.pyを指定します。
- inputsのsourceには入力ファイルがあるS3のPATH、destinationにはコンテナ上でそのファイルを置く場所を指定します。
- outputsのsourceには出力したいファイルのコンテナ上の場所、destinationには出力したいファイルを置くS3のPATHを指定します。
- argumentsには、任意でスクリプト（preprocess.py）内に渡す引数を設定できます。今回はTargetEncodingに使用するターゲット変数の名前やTargetEncodingする変数名、ハイパーパラメータなどを設定していますが、これらはpreprocess.py内に直接記述することも可能です。

また、inputs、outputsは必ずしも設定する必要はなく、preprocess.py内でインターネット上からファイルを取得して、何かしらの処理をして出力することもできますし、入力ファイルから機械学習モデルを学習し、ひとまず精度を確認したり、学習用インスタンスで実行するためのコードを開発するといったことにも使えます（このnotebookでは前処理・特徴量作成のあとにXGBoostモデルの学習も行っています）。

In [ ]:
%%time

rapids_processor.run(
    code="preprocess.py",
    inputs=[
        ProcessingInput(source=input_train, destination='/opt/ml/processing/input/train'), 
        ProcessingInput(source=input_validation, destination='/opt/ml/processing/input/valid')
    ], 
    outputs=[
        ProcessingOutput(source='/opt/ml/processing/output/train', destination=output_s3_path),
        ProcessingOutput(source='/opt/ml/processing/output/valid', destination=output_s3_path)
    ],
    arguments=[
        'TARGET_COL', 'income',
        'TE_COLS', 'class of worker, education, major industry code',
        'SMOOTH', '0.001',
        'SPLIT', 'interleaved',
        'FOLDS', '5'
    ],
    wait=True,
    logs=True,
    job_name=None,
    experiment_config=None, 
    kms_key=None
)

### 処理結果の確認

s3のバケットからファイルを呼び出し、処理後のファイルを確認してみましょう

In [ ]:
obj_list=s3.list_objects_v2(Bucket=bucket, Prefix=prefix + '/input/dataset/')

file=[]
for contents in obj_list['Contents']:
    file.append(contents['Key'])
print(file)

In [ ]:
file_path = file[file.index(prefix + '/input/dataset/train.csv')]

In [ ]:
import io

# oldの特徴量データを読み込む
response = s3.get_object(Bucket=bucket, Key=file_path)
response_body = response["Body"].read()
train_df = pd.read_csv(io.BytesIO(response_body), header=0, delimiter=",", low_memory=False)

In [ ]:
# カテゴリ変数がLabel encodingされていることと指定したカテゴリ変数がTarget encodingされていることを確認できました。
train_df.head()

### 後片付け

SageMaker ProcessingはJobが完了するとインスタンスが自動的に停止し、削除されます。

一方、このノートブックインスタンスについてはSageMakerのコンソールから、手動で停止しない限り、料金が継続的にかかってしまいます。

このノートブックの実行が終わったらインスタンスの停止と削除をお願いいたします。